In [1]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $500000$) and make a hash table from the $a_p$ for $50$ primes (the first $50$ over $500000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [2]:
try:
    hashtab_11_50 = load('hashtab11_50')
except IOError:
    hashtab_11_50 = make_hash(11,1,500000,50)
    hashtab_11_50 = dict([(k,v) for k,v in hashtab_11_50.items() if len(v)>1])
    save(hashtab_11_50, 'hashtab11_50')
len(hashtab_11_50)    

731

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{11}$ for all $30$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [3]:
congruent_sets = [s for s in hashtab_11_50.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 731
Distribution of sizes of nontrivial congruent sets: Counter({2: 731})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors).  With the default mumax=5000000 there is one pair for which this is not rigorous:  '49a1' and '78400gw1'.

In [4]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(11,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,11,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 11-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 11-congruent".format(len(congruent_sets)))

all 731 sets of curves really are 11-congruent


### So we have no "false positive" pairs

### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [5]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 731 pairs, there are only 100 distinct pairs of j-invariants


In [6]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

100 congruent pairs up to twist: [['11466k1', '126126bj1'], ['383040ip1', '383040ir1'], ['78650w1', '78650bc1'], ['80325bo1', '80325bp1'], ['8550d1', '111150bl1'], ['645f1', '70305g1'], ['1254b1', '264594e1'], ['286650nv1', '286650oc1'], ['22050dx1', '374850pj1'], ['307626j1', '307626k1'], ['187690d1', '187690e1'], ['2755b1', '476615d1'], ['8640bc1', '319680cn1'], ['13104q1', '13104r1'], ['263578b1', '263578c1'], ['221130cd1', '221130cf1'], ['8120d1', '332920k1'], ['7830f1', '477630e1'], ['445578o1', '445578p1'], ['208080u1', '208080bb1'], ['457380q1', '457380t1'], ['1115a1', '125995a1'], ['148225ck1', '148225cl1'], ['18837e1', '239499f1'], ['1782b1', '447282e1'], ['432450er1', '432450es1'], ['1296d1', '9072i1'], ['31746q1', '222222bb1'], ['15600bw1', '265200ce1'], ['494018d1', '494018i1'], ['62424d1', '436968bs1'], ['202400r1', '349600y1'], ['513b1', '77463b1'], ['4275a1', '175275y1'], ['101568z1', '101568ba1'], ['43632s1', '479952t1'], ['7830p1', '477630j1'], ['1904d1', '36176v1'], [

### Test whether all these are irreducible, i.e. have no $11$-isogenies:

In [7]:
for s2 in congruent_pairs:
    for s in s2:
        E = EllipticCurve(s)
        assert not E.isogenies_prime_degree(11)

#### In fact the only isogeny degrees among these curves are 2, 3, 7:

In [8]:
[legendre_symbol(d,11) for d in [2,3,7]]

[-1, 1, -1]

In [9]:
sum([Set(EllipticCurve(s[0]).reducible_primes()) for s in congruent_pairs],Set())

{2, 3, 7}

In [10]:
[tuple(EllipticCurve(si).reducible_primes() for si in s) for s in congruent_pairs]

[([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([7], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([3], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([3], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([], []),
 ([3], []),
 ([2], []),
 ([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([2], []),
 ([], []),
 ([2], []),
 ([3], []),
 ([], []),
 ([], []),
 ([2, 3], []),
 ([], []),
 ([], []),
 ([3], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([3], []),
 ([2], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 

In [10]:
out = open("mod11pairs.m",'w')
out.write('pairs := [\\\n')
for s in congruent_sets[:-1]:
    out.write('["{}", "{}"],\\\n'.format(s[0],s[1]))
s = congruent_sets[-1]
out.write('["{}", "{}"]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [11]:
out = open("mod11pairs_uptotwist.m",'w')
out.write('pairs := [\\\n')
for s in congruent_pairs[:-1]:
    out.write('[EllipticCurve("{}"), EllipticCurve("{}")],\\\n'.format(s[0],s[1]))
s = congruent_pairs[-1]
out.write('[EllipticCurve("{}"), EllipticCurve("{}")]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [11]:
len(congruent_pairs)

100

In [12]:
for pair in congruent_sets:
    sizes = [len(EllipticCurve(lab).isogeny_class()) for lab in pair]
    if sizes != [1,1]:
        print("pair {}: class sizes {}".format(pair,sizes))

pair ['1904d1', '36176v1']: class sizes [2, 1]
pair ['22050dx1', '374850pj1']: class sizes [2, 1]
pair ['7350j1', '124950bg1']: class sizes [2, 1]
pair ['4275a1', '175275y1']: class sizes [2, 1]
pair ['700h1', '11900g1']: class sizes [2, 1]
pair ['2352l1', '39984br1']: class sizes [2, 1]
pair ['6300z1', '107100cn1']: class sizes [2, 1]
pair ['1666d1', '31654h1']: class sizes [2, 1]
pair ['3850x1', '65450bh1']: class sizes [2, 1]
pair ['3174d1', '403098l1']: class sizes [4, 1]
pair ['28224en1', '479808my1']: class sizes [2, 1]
pair ['26950db1', '458150ek1']: class sizes [2, 1]
pair ['26950bh1', '458150bh1']: class sizes [2, 1]
pair ['11200bb1', '190400dh1']: class sizes [2, 1]
pair ['28224x1', '479808ju1']: class sizes [2, 1]
pair ['25200by1', '478800fd1']: class sizes [2, 1]
pair ['4275c1', '175275l1']: class sizes [2, 1]
pair ['78650w1', '78650bc1']: class sizes [2, 1]
pair ['12600ba1', '239400r1']: class sizes [2, 1]
pair ['84966ct1', '84966cu1']: class sizes [2, 1]
pair ['6300s1', '

In [13]:
[legendre_symbol(a,11) for a in [2,3,7]]

[-1, 1, -1]

In [15]:
npc=0
out = open("mod11_pairs.m", "w")
out.write('pairs := [\\\n')
for i,pair in enumerate(congruent_pairs):
    cc = [all_c(c) for c in pair]
    for c1 in cc[0]:
        for c2 in cc[1]:
            npc+=1
            out.write('["{}", "{}"]'.format(c1[0],c2[0]))
            if i+1<len(congruent_pairs):
                out.write(',')
            out.write('\\\n')
out.write('];\n')
out.close()           
print("{} pairs written to file".format(npc))            

119 pairs written to file


In [16]:
len(congruent_pairs)

100

In [1]:
E1 = EllipticCurve('25921a1')
E2 = EllipticCurve('78400gw1')

In [12]:
L = [next_prime(400000)]
while len(L)<36:
    L.append(next_prime(L[-1]))
 

In [13]:
all((E1.ap(p)==E2.ap(p)) for p in L)

False

In [15]:
p=L[-1]

In [17]:
E1.ap(p), E2.ap(p)

(-586, 586)

In [18]:
586.factor()

2 * 293

In [14]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [15]:
irred_js = Set()
for s in congruent_sets:
    js = set_js(s)
    irred_js = irred_js + Set(js)

In [16]:
len(irred_js)

212